In [1]:
import json
import requests
import pandas as pd
from DATA.CODE_REFERENCES.ENVIRONMENT.compliance_env import get_compliance_data, check_valid_dates
from datetime import datetime, timedelta

In [2]:
company = "rudin"
building = "845_third"
## Notes for date
## an example of calculating yesterday's date, always dynamically change the date
yesterday = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')
start_date = yesterday
end_date =  yesterday
## compliance is to check whether past recommendations have been accepted, 
## therefore no weekends, no holidays, only past days' data, no today or future data
## should check whether the date has data
# start_date and end_date are both included, returned a dict of {date: has_data: bool}
days_valid =  check_valid_dates(start_date, end_date, company, building)
valid_days_list = [date.strftime('%Y-%m-%d') for date, has_data in days_valid.items() if has_data]
print("days with data: ", valid_days_list)


days with data:  ['2024-08-15']


In [4]:
if len(valid_days_list)>0:
    for date in valid_days_list:
        df = get_compliance_data(company, building, date, ecm_command_type='all')
df.head()

,date,ecm_command_type,complied
0,2024-08-15,startup,False
1,2024-08-15,shutdown,False
2,2024-08-15,lunch_rampdown,True
3,2024-08-15,lunch_rampup,True
4,2024-08-15,final_rampdown,True


In [9]:
## calculate compliance rate
df = df.dropna(subset=['ecm_command_type'])
## total compliance rate
print(f"total compliance rate: {df['complied'].sum() / len(df)}")
## give a breakdown of compliance rate by ecm_command_type
print(df.groupby('ecm_command_type')['complied'].sum()/df.groupby('ecm_command_type')['ecm_command_type'].count())

total compliance rate: 0.5
ecm_command_type
final_rampdown    1.0
lunch_rampdown    1.0
lunch_rampup      1.0
shutdown          0.0
startup           0.0
dtype: object
